# IMAP Data Access API

The IMAP SDC provides a Data Access API to **query**, **download**, and **upload** data. There are three ways to use the API:

1. The `imap-data-access` command-line utility
2. The `imap_data_access` python package
3. Calls to the REST API directly

This notebook provies instructions and examples of how to use the command-line utility and python package. *For information on how to use the REST API directly, see https://imap-processing.readthedocs.io/en/latest/data-access-api/index.html.*

## Data Directory

Before using the API, it is important to note that the folder structure for data files within the IMAP SDC is rigidly defined, so the data access will mimic that structure to make sure all data is stored in the same heirarchical structure as the SDC. This will enable seamless transition between a user's local system and the SDC.

The directory structure is as follows:

```
<IMAP_DATA_DIR>/
  imap/
    <instrument>/
      <data_level>/
        <year>/
          <month>/
            <filename>
```

for example:

```
/data/
  imap/
    swe/
      l0/
        2024/
          01/
            imap_swe_l0_sci_20240105_v001.pkts
```

By default, the `<IMAP_DATA_DIR>` is the user's current working directory. However, this can be changed by setting the `IMAP_DATA_DIR` environment variable:

In [1]:
!mkdir -p /Users/mabo8927/imap_data/
%env IMAP_DATA_DIR=/Users/mabo8927/imap_data
!echo $IMAP_DATA_DIR

env: IMAP_DATA_DIR=/Users/mabo8927/imap_data
/Users/mabo8927/imap_data


## Installation

The API can be installed via `pip`:

In [2]:
!pip install imap-data-access
!imap-data-access --version

  Using cached imap_data_access-0.7.0-py3-none-any.whl
imap-data-access 0.7.0


We leave it as an exercise to the user to install the package into their software environment of choice.

## Command Line Utility

The following section shows examples of how to use the API via the `imap-data-access` command line utility.

In [3]:
# Show help documentation
!imap-data-access -h

usage: imap-data-access [-h] [--version] [--api-key API_KEY]
                        [--data-dir DATA_DIR] [--url URL] [--debug] [-v]
                        {download,query,upload} ...

This command line program accesses the IMAP SDC APIs to query, download, and
upload data files.

positional arguments:
  {download,query,upload}
    download            Download a file from the IMAP SDC to the locally
                        configured data directory.
    query               Query the IMAP SDC for files matching the query
                        parameters. The query parameters are optional, but at
                        least one must be provided.
    upload              Upload a file to the IMAP SDC. This must be the full
                        path to the file. E.g.
                        imap/mag/l0/2025/01/imap_mag_l0_raw_20250101_v001.pkts

optional arguments:
  -h, --help            show this help message and exit
  --version             show program's version number and exit

#### Querying

In [4]:
# Show query help documentation
!imap-data-access query -h

usage: imap-data-access query [-h]
                              [--instrument {codice,glows,hi,hit,idex,lo,mag,swapi,swe,ultra}]
                              [--data-level DATA_LEVEL]
                              [--descriptor DESCRIPTOR]
                              [--start-date START_DATE] [--end-date END_DATE]
                              [--repointing REPOINTING] [--version VERSION]
                              [--extension EXTENSION]
                              [--output-format {table,json}]

Query the IMAP SDC for files matching the query parameters. The query
parameters are optional, but at least one must be provided.

optional arguments:
  -h, --help            show this help message and exit
  --instrument {codice,glows,hi,hit,idex,lo,mag,swapi,swe,ultra}
                        Name of the instrument
  --data-level DATA_LEVEL
                        Data level of the product (l0, l1a, l2, etc.)
  --descriptor DESCRIPTOR
                        Descriptor of the produ

In [5]:
# Query with a single parameter
!imap-data-access query --instrument glows

Found [7] matching files
-----------------------------------------------------------------------------------------------------------------|
Instrument|Data Level|Descriptor|Start Date|Repointing|Version|Filename                                          |
-----------------------------------------------------------------------------------------------------------------|
glows     |l0        |raw       |20110921  |          |v001   |imap_glows_l0_raw_20110921_v001.pkts              |
glows     |l1a       |de        |20110920  |          |v001   |imap_glows_l1a_de_20110920_v001.cdf               |
glows     |l1a       |hist      |20110920  |          |v001   |imap_glows_l1a_hist_20110920_v001.cdf             |
glows     |l1a       |hist      |20110921  |          |v001   |imap_glows_l1a_hist_20110921_v001.cdf             |
glows     |l1b       |de        |20110920  |          |v001   |imap_glows_l1b_de_20110920_v001.cdf               |
glows     |l1b       |hist      |20110920  |          |

In [6]:
# Query with multiple parameters
!imap-data-access query --instrument glows --data-level l1a --descriptor hist

Found [2] matching files
-----------------------------------------------------------------------------------------------------------------|
Instrument|Data Level|Descriptor|Start Date|Repointing|Version|Filename                                          |
-----------------------------------------------------------------------------------------------------------------|
glows     |l1a       |hist      |20110920  |          |v001   |imap_glows_l1a_hist_20110920_v001.cdf             |
glows     |l1a       |hist      |20110921  |          |v001   |imap_glows_l1a_hist_20110921_v001.cdf             |
-----------------------------------------------------------------------------------------------------------------|


In [7]:
# Change the output format
!imap-data-access query --instrument glows --data-level l1a --descriptor hist --output-format json

[{'file_path': 'imap/glows/l1a/2011/09/imap_glows_l1a_hist_20110920_v001.cdf', 'instrument': 'glows', 'data_level': 'l1a', 'descriptor': 'hist', 'start_date': '20110920', 'repointing': None, 'version': 'v001', 'extension': 'cdf', 'ingestion_date': '2024-07-09 15:42:23'}, {'file_path': 'imap/glows/l1a/2011/09/imap_glows_l1a_hist_20110921_v001.cdf', 'instrument': 'glows', 'data_level': 'l1a', 'descriptor': 'hist', 'start_date': '20110921', 'repointing': None, 'version': 'v001', 'extension': 'cdf', 'ingestion_date': '2024-07-09 15:42:24'}]


#### Download a file

In [8]:
# Show download help documentation
!imap-data-access download -h

usage: imap-data-access download [-h] file_path

Download a file from the IMAP SDC to the locally configured data directory.

positional arguments:
  file_path   This must be the full path to the file. E.g.
              imap/mag/l0/2025/01/imap_mag_l0_raw_20250101_v001.pkts

optional arguments:
  -h, --help  show this help message and exit


In [9]:
# Download a file
!imap-data-access download imap_glows_l1a_hist_20110921_v001.cdf

# Check to see that the file exists
!ls -lt $IMAP_DATA_DIR/imap/glows/l1a/2011/09/imap_glows_l1a_hist_20110921_v001.cdf

Successfully downloaded the file to: /Users/mabo8927/imap_data/imap/glows/l1a/2011/09/imap_glows_l1a_hist_20110921_v001.cdf
-rw-r--r--  1 mabo8927  2260  8183501 Jul 31 16:43 /Users/mabo8927/imap_data/imap/glows/l1a/2011/09/imap_glows_l1a_hist_20110921_v001.cdf


#### Upload a file

In [10]:
# Show upload help documentation
!imap-data-access upload -h

usage: imap-data-access upload [-h] file_path

Upload a file to the IMAP SDC. This must be the full path to the file. E.g.
imap/mag/l0/2025/01/imap_mag_l0_raw_20250101_v001.pkts

positional arguments:
  file_path   This must be the full path to the file. E.g.
              imap/mag/l0/2025/01/imap_mag_l0_raw_20250101_v001.pkts

optional arguments:
  -h, --help  show this help message and exit


In [11]:
# Show the current CoDICE files to see what is there
!imap-data-access query --instrument codice --descriptor hskp

Found [12] matching files
-----------------------------------------------------------------------------------------------------------------|
Instrument|Data Level|Descriptor|Start Date|Repointing|Version|Filename                                          |
-----------------------------------------------------------------------------------------------------------------|
codice    |l0        |hskp      |20240610  |          |v001   |imap_codice_l0_hskp_20240610_v001.pkts            |
codice    |l0        |hskp      |20240610  |          |v100   |imap_codice_l0_hskp_20240610_v100.pkts            |
codice    |l0        |hskp      |20240610  |          |v200   |imap_codice_l0_hskp_20240610_v200.pkts            |
codice    |l0        |hskp      |20240610  |          |v210   |imap_codice_l0_hskp_20240610_v210.pkts            |
codice    |l0        |hskp      |20240720  |          |v001   |imap_codice_l0_hskp_20240720_v001.pkts            |
codice    |l0        |hskp      |20240721  |          

In [12]:
# Upload the file
!imap-data-access upload $IMAP_DATA_DIR/imap_codice_l0_hskp_20240725_v001.pkts

Successfully uploaded the file to the IMAP SDC


In [13]:
# Check the CoDICE files to see that it was uploaded
!imap-data-access query --instrument codice --descriptor hskp

Found [13] matching files
-----------------------------------------------------------------------------------------------------------------|
Instrument|Data Level|Descriptor|Start Date|Repointing|Version|Filename                                          |
-----------------------------------------------------------------------------------------------------------------|
codice    |l0        |hskp      |20240610  |          |v001   |imap_codice_l0_hskp_20240610_v001.pkts            |
codice    |l0        |hskp      |20240610  |          |v100   |imap_codice_l0_hskp_20240610_v100.pkts            |
codice    |l0        |hskp      |20240610  |          |v200   |imap_codice_l0_hskp_20240610_v200.pkts            |
codice    |l0        |hskp      |20240610  |          |v210   |imap_codice_l0_hskp_20240610_v210.pkts            |
codice    |l0        |hskp      |20240720  |          |v001   |imap_codice_l0_hskp_20240720_v001.pkts            |
codice    |l0        |hskp      |20240721  |          

#### Bonus: Let's see it in action in AWS

In [14]:
!imap-data-access upload $IMAP_DATA_DIR/imap_codice_l0_lo-counters-singles_20240429_v001.pkts

Successfully uploaded the file to the IMAP SDC


## Python Package
The following section shows examples of how to use the API via the `imap_data_access` Python package.

In [15]:
# import the package
import imap_data_access

The `<IMAP_DATA_DIR>` can be set via the `config` dictionary:

In [16]:
from pathlib import Path
imap_data_access.config["DATA_DIR"] = Path.home() / "imap_data"

#### Querying

In [17]:
results = imap_data_access.query(instrument="mag", data_level="l0")
for result in results:
    print(result)

{'file_path': 'imap/mag/l0/2025/05/imap_mag_l0_raw_20250502_v000.pkts', 'instrument': 'mag', 'data_level': 'l0', 'descriptor': 'raw', 'start_date': '20250502', 'repointing': None, 'version': 'v000', 'extension': 'pkts', 'ingestion_date': '2024-07-16 10:27:13'}
{'file_path': 'imap/mag/l0/2025/05/imap_mag_l0_raw_20250502_v001.pkts', 'instrument': 'mag', 'data_level': 'l0', 'descriptor': 'raw', 'start_date': '20250502', 'repointing': None, 'version': 'v001', 'extension': 'pkts', 'ingestion_date': '2024-07-16 10:29:33'}
{'file_path': 'imap/mag/l0/2025/05/imap_mag_l0_raw_20250511_v000.pkts', 'instrument': 'mag', 'data_level': 'l0', 'descriptor': 'raw', 'start_date': '20250511', 'repointing': None, 'version': 'v000', 'extension': 'pkts', 'ingestion_date': '2024-07-19 16:34:35'}
{'file_path': 'imap/mag/l0/2025/05/imap_mag_l0_raw_20250511_v001.pkts', 'instrument': 'mag', 'data_level': 'l0', 'descriptor': 'raw', 'start_date': '20250511', 'repointing': None, 'version': 'v001', 'extension': 'pkts

#### Download a file

In [18]:
file_path = imap_data_access.download("imap_mag_l0_raw_20250502_v001.pkts")
print(file_path)

/Users/mabo8927/imap_data/imap/mag/l0/2025/05/imap_mag_l0_raw_20250502_v001.pkts


#### Upload a file

In [19]:
imap_data_access.upload(f"{Path.home()}/imap_data/imap_codice_l0_hskp_20240726_v001.pkts")

In [20]:
# Check to see if file was uploaded
results = imap_data_access.query(instrument="codice", descriptor="hskp", start_date="20240726", end_date="20240727")
print(results)

[{'file_path': 'imap/codice/l0/2024/07/imap_codice_l0_hskp_20240726_v001.pkts', 'instrument': 'codice', 'data_level': 'l0', 'descriptor': 'hskp', 'start_date': '20240726', 'repointing': None, 'version': 'v001', 'extension': 'pkts', 'ingestion_date': '2024-08-01 22:27:27'}]


#### Bonus: validate/construct a file path

In [23]:
science_file = imap_data_access.ScienceFilePath("imap_mag_l0_raw_20250502_v001.pkts")
print(science_file.start_date)
print(science_file.descriptor)

filepath = science_file.construct_path()
print(filepath)

20250502
raw
/Users/mabo8927/imap_data/imap/mag/l0/2025/05/imap_mag_l0_raw_20250502_v001.pkts


## Future work

The SDC continues to develop the API for the greater needs of the IMAP community. We currently have plans to add the following features (subject to change):

- Add `filename` as a query parameter to return information for a particular file
- Add `today` as a query parameter to return list of files that were acquired in the last 24 hours
- Add `days_ago=N` as a query parameter to return a list of file that were acquired in the last `N` days
- Add ability to upload ancillary files

## Ideas/requests for other API features or endpoints?

We would like to gather user feedback on how we could improve or expand upon the existing API. What API endpoints could we add? What kind of information would be useful to get from the API?

For feature requests and bug reports, please open a [new issue](https://github.com/IMAP-Science-Operations-Center/imap-data-access/issues/new) in the Data Access API repository.